<a href="https://colab.research.google.com/github/softplus/detweet/blob/main/tweets_to_csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Copyright 2023-2024, John Mueller
# MIT License, see LICENSE file
# SPDX-FileCopyrightText: 2023-2024 John Mueller
# SPDX-License-Identifier: MIT
# https://github.com/softplus/detweet

# Settings
your_username = "johnmu"
tweet_path = 'twitter-2023-12-24-8dfabcdf6c76db27c447cb87ed35e1ecd5a9bc83b89460f98d55dfb97ed4abba'


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# set local paths -- in Google Drive
tweet_file_raw   = '/content/drive/MyDrive/' + tweet_path + '/data/tweets.js' # normal archive file
tweet_file_csv = '/content/drive/MyDrive/' + tweet_path + '/tweets.csv' # output file


In [ ]:
your_username = your_username or "i"
import json, re, datetime

# clean up tweets.js file
line_count = 0
tweet_count = 0
file_out = open(tweet_file_csv, "w")
fields = ["created_at", "tweet_date", "id_str", "in_reply_to_status_id_str", "in_reply_to_screen_name",
          "favorite_count", "retweet_count", "full_text", "just_text", "tweet_url"]
# url = https://twitter.com/i/status/1737186036147528084 or w/username
file_out.write(("\t".join(fields)) + "\n")
row = {}
with open(tweet_file_raw, "r") as file_in:
  for line in file_in:
    if '"tweet" :' in line:
      if row:
        row["tweet_url"] = "https://twitter.com/%s/status/%s" % (your_username, row["id_str"])
        row["just_text"] = re.sub(r'@[^ ]+ ', '', row["full_text"]).strip()
        timestamp_dt = datetime.datetime.strptime(row["created_at"], '%a %b %d %X %z %Y')
        row["tweet_date"] = timestamp_dt.strftime("%Y-%m-%dT%H:%M:%S")

        file_out.write("\t".join([(x in row and row[x] or "") for x in fields]))
        file_out.write("\n")
        row = {}
        tweet_count += 1
    for x in fields:
      if '"%s" :' % x in line: # much hacky, parsing JSON by field, saves you from figuring out escaping
        s = line.strip()
        if (s[-1]==","): s=s[:-1]
        item = json.loads("{" + s + "}")
        row[x] = item[x]
        while "\n" in row[x]: row[x] = row[x].replace("\n", " ") # remove linebreaks
    line_count+= 1
    if (line_count % 10000 ==0): print("line: %d" % line_count)

if row: # remainder
  file_out.write("\t".join([(x in row and row[x] or "") for x in fields]))
file_out.close()

print("Done. Processed %d lines, %d tweets" % (line_count, tweet_count))